In [42]:
import pandas as pd
import sqlite3
from datetime import datetime
import os

df = pd.read_csv('region.csv')
df = df[['Country', 'GDP_USD_billions', 'Region']]   

# 현재 날짜 확인
now = datetime.now()
current_year = now.year
half_year = '_1' if now.month < 7 or (now.month == 7 and now.day == 1) else '_2'
current_period = f"Gdp{current_year}{half_year}"

db_filename = 'gdp.db'
conn = sqlite3.connect(db_filename)
cursor = conn.cursor()

# 테이블 생성
cursor.execute(f'''
CREATE TABLE IF NOT EXISTS {current_period} (
    Country TEXT PRIMARY KEY,
    GDP_USD_billion REAL,
    YEAR TEXT
)
''')

# YYYY-2 데이터가 있는지 확인
cursor.execute(f'''
SELECT COUNT(*)
FROM {current_period}
WHERE YEAR = ?
''', (f"{current_year}_2",))
count = cursor.fetchone()[0]

# YYYY-2 데이터가 없는 경우에 데이터 삽입
if count == 0:
    for _, row in df.iterrows():
        cursor.execute(f'''
        INSERT OR REPLACE INTO {current_period} (Country, GDP_USD_billion, YEAR)
        VALUES (?, ?, ?)
        ''', (row['Country'], row['GDP_USD_billions'], current_period[3:]))

print(f"데이터베이스 저장 {db_filename}")

conn.commit()
conn.close()

데이터베이스 저장 gdp.db


In [44]:
conn = sqlite3.connect(db_filename)
cursor = conn.cursor()
query = f"""
SELECT * FROM {current_period}
"""

df = pd.read_sql_query(query, conn)
display(df)

conn.close()

,Country,GDP_USD_billion,YEAR
0,United States,28781.08,2024_2
1,China,18532.63,2024_2
2,Germany,4591.10,2024_2
3,Japan,4110.45,2024_2
4,India,3937.01,2024_2
...,...,...,...
189,Kiribati,0.31,2024_2
190,Palau,0.31,2024_2
191,Marshall Islands,0.30,2024_2
192,Nauru,0.16,2024_2
